<h1>GRADUATION PROJECT<h1>
FINAL STAGE 

THIS STAGE SHOW HOW PLATFORM INTERFACE WORKS
<ul><li>
    Collects title, synopsis, problem statement, tags and industry of the case study from the mentee </li>
    <li>
    Presents UX tips based on the title, synopsis, problem statement, tags of the case study by cosine similarity </li>
    <li>
    Mentee selects mentor from the mentor list. List comes to the mentee according to the industry </li>

In [62]:
import sklearn as sk
import pandas as pd
import numpy as np
import json
import random
from sklearn.model_selection import train_test_split #I need this to split the data
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import re

industries = pd.read_csv("industries.csv")
tags = pd.read_csv("tags.csv")
sessionnotes = pd.read_csv("sessionnotes.csv", encoding = "utf_8_sig", sep =";")
synopsis_data = pd.read_csv("data.csv")

In [63]:
def isInputValid(_input, _range):
    if not _input.isnumeric():
        print("Please enter a numeric input")
        return False
    elif int(_input) >_range or int(_input) < 1:
        print("Please enter a number in range 1-{}".format(len(_range)))
        return False
    else:
        return True 

In [64]:
def getSessionNote(df, df_sn, synopsis):
    text = list(df['data'].values.astype('U')) + [synopsis]
    vect = CountVectorizer(stop_words='english') 
    #I fit the model with the words from the review text
    vect = vect.fit(text) 
    # make a matrix
    docu_feat = vect.transform(text) 
    cosine_sim = linear_kernel(docu_feat,docu_feat)
    scores = list(enumerate(cosine_sim[len(df)]))
    scores = sorted(scores, key = lambda x:x[1], reverse = True)
    scores = scores[1:3]  #session note itself is considered as 0, 1 and 2 are considered as the most similar ones but there are seen by the users regardless of their voting ( user ticks).
    indices = [i[0] for i in scores] # here scores includes 2 elements. 1 and 2. The most two similar ones.
    for i in range(3,len(scores)):
        if not df_sn.at[df.at[scores[i],"sessionnote"],"isRated"] and score[i] > 0.5: #threshold for suggestions that is not seen from anyone on the platform but also it should meet the similarity rate by 0.5
            indices.append(i)
            break
    return df_sn["notes"].iloc[list(df["sessionnote"].iloc[indices])]

In [65]:
def listMentors(industry): #this is for the list of mentors showed to the mentee before the booking
    counter = 0
    with open("mentors.json") as infile:
        mentors = json.load(infile)
    for mentor in mentors:
        if (industry in mentor["industry"]):
            print("{}-) ".format(counter) + mentor["name"])
            counter += 1

In [66]:
def getMenteeData(synopsis_data, sessionnotes):
    
    print("Which industry is related to your case study.")
    for i  in range(len(industries)):
        print("{}-) {}".format(i+1,industries.at[i,"ind"]))
    selection = input("Selection: ")
    
    if(isInputValid(selection, len(industries))):
        industry_index = int(selection)-1
    
    sub_tags = tags.where(tags["industry"] == industry_index).dropna()
    indices = list(sub_tags.dropna().index)
    sub_tags = sub_tags.reset_index()
    print("\n\nTags suggestions(At least 1, at most 3).")
    for i in range(len(sub_tags)):
        print("#{}".format(sub_tags.at[i,"tags"])) #element at ith index and tags column 
    tag_list = []
    
    i = 0
    while(i < 3):
        tag = input("Enter {}. tag: ".format(i+1))
        if(tag == "" and i != 0):
            break
        else:
            tag_list.append(tag)
            i+=1
    title = input("Title of the case study: ")
    synopsis = input("Synopsis: ")
    problem_statement = input("Problem statement: ")
    
    data = synopsis + " " + title +"" + problem_statement
    
    for tag in tag_list:
        data += " " + tag
    
    recommendations = getSessionNote(synopsis_data, sessionnotes, data) 
    
    print("You have {} UX Tips according your case study! These are gifts from other mentees ".format(len(recommendations)))
    for recommendation in recommendations:
        print(recommendation)
        
    print("\nDirected to your whiteboard...Loading")
    
    print("\nListing mentors...")
    
    with open("mentors.json") as infile:
        mentors = json.load(infile)
    
    listMentors(industries.at[industry_index,"ind"])
    input("Select a mentor: ")
    
    print("Book a session for your selected mentor...")
    session_note = input("Save your session notes for collaboration:\n")
    tag_indices = []
    for i in range(3):
        if len(tag_list) > i:
            tag_indices.append(tags["tags"].where(tags["tags"] == tag_list[i]).dropna().index[0])
        else:
            tag_indices.append(-1)
            
    new_data = { "data":data,
                 "industry": industry_index,
                 "tag":tag_indices[0],
                 "tag2":tag_indices[1],
                 "tag3":tag_indices[2],
                 "sessionnote":len(sessionnotes)}
    
    new_data = pd.DataFrame(data = new_data, index = [len(synopsis_data)])
    synopsis_data = synopsis_data.append(new_data)
    synopsis_data.to_csv("data.csv", index=False)
   
    #when the users add a new session note after a mentoring session, the rating of the session notes are zero and rated false
    new_sn = {
        "notes": session_note,
        "rating_design": 0,
        "rating_uxresearch": 0,
        "rating_writing": 0,
        "rating_presentation": 0,
        "isRated": False
    }
    new_sn = pd.DataFrame(data = new_sn, index = [len(sessionnotes)])
    sessionnotes = sessionnotes.append(new_sn)
    sessionnotes.to_csv("sessionnotes.csv", encoding="utf_8_sig", index=False)
    
    print("Session is ended.\n Saving data...")

## When the mentee enters her case study information (title, synopsis, problem statement and tags), she will receive suggestions according to cosine similarity algorithm. 


In [ ]:
getMenteeData(synopsis_data, sessionnotes)

Which industry is related to your case study.
1-) communication
2-) e-commerce
3-) education
4-) energy&earth
5-) entertainment
6-) finance
7-) food and beverage &consumption
8-) health &psychology
9-) tech
10-) travel&transportation
